In [1]:
import pandas as pd

In [ ]:
df1=pd.read_csv("/path/to/ARPA_Milan_CO_2024.csv")
df2 = pd.read_csv("/path/to/ARPA_Milan_CO_2019_2023.csv")

In [3]:
df1

,Unnamed: 0,idsensore,data,valore
0,95,5812,2024-06-03 01:00:00,0.2
1,96,5812,2024-06-03 02:00:00,0.2
2,97,5812,2024-06-03 03:00:00,0.2
3,98,5812,2024-06-03 04:00:00,0.2
4,99,5812,2024-06-03 05:00:00,0.2
...,...,...,...,...
43004,2572624,5835,2024-05-19 20:00:00,0.2
43005,2572625,5835,2024-05-19 21:00:00,0.3
43006,2572626,5835,2024-05-19 22:00:00,0.2
43007,2572627,5835,2024-05-19 23:00:00,0.3


In [4]:
df1 = df1.drop(columns=['Unnamed: 0'])
df1

,idsensore,data,valore
0,5812,2024-06-03 01:00:00,0.2
1,5812,2024-06-03 02:00:00,0.2
2,5812,2024-06-03 03:00:00,0.2
3,5812,2024-06-03 04:00:00,0.2
4,5812,2024-06-03 05:00:00,0.2
...,...,...,...
43004,5835,2024-05-19 20:00:00,0.2
43005,5835,2024-05-19 21:00:00,0.3
43006,5835,2024-05-19 22:00:00,0.2
43007,5835,2024-05-19 23:00:00,0.3


In [5]:
df2

,Unnamed: 0,idsensore,data,valore
0,0,5812,2019-01-01 00:00:00,0.6
1,1,5812,2019-01-01 01:00:00,0.6
2,2,5812,2019-01-01 02:00:00,0.7
3,3,5812,2019-01-01 03:00:00,0.7
4,4,5812,2019-01-01 04:00:00,0.8
...,...,...,...,...
208825,216548,5835,2023-12-31 19:00:00,0.3
208826,216549,5835,2023-12-31 20:00:00,0.4
208827,216550,5835,2023-12-31 21:00:00,0.5
208828,216551,5835,2023-12-31 22:00:00,0.5


In [6]:
df2 = df2.drop(columns=['Unnamed: 0'])

In [7]:
df2

,idsensore,data,valore
0,5812,2019-01-01 00:00:00,0.6
1,5812,2019-01-01 01:00:00,0.6
2,5812,2019-01-01 02:00:00,0.7
3,5812,2019-01-01 03:00:00,0.7
4,5812,2019-01-01 04:00:00,0.8
...,...,...,...
208825,5835,2023-12-31 19:00:00,0.3
208826,5835,2023-12-31 20:00:00,0.4
208827,5835,2023-12-31 21:00:00,0.5
208828,5835,2023-12-31 22:00:00,0.5


In [8]:
# Combine the two DataFrames vertically
CO_df = pd.concat([df1, df2], axis=0, ignore_index=True)
CO_df = CO_df.drop_duplicates()
CO_df = CO_df.sort_values(by=['idsensore', 'data']).reset_index(drop=True)
CO_df

,idsensore,data,valore
0,5812,2019-01-01 00:00:00,0.6
1,5812,2019-01-01 01:00:00,0.6
2,5812,2019-01-01 02:00:00,0.7
3,5812,2019-01-01 03:00:00,0.7
4,5812,2019-01-01 04:00:00,0.8
...,...,...,...
251834,5835,2024-12-31 19:00:00,1.0
251835,5835,2024-12-31 20:00:00,1.4
251836,5835,2024-12-31 21:00:00,1.5
251837,5835,2024-12-31 22:00:00,2.0


In [9]:
CO_df['data'] = pd.to_datetime(CO_df['data'])

In [10]:
CO_df['date'] = CO_df['data'].dt.date
CO_df['hour'] = CO_df['data'].dt.hour

# Filter data 
filtered = CO_df[(CO_df['hour'] >= 21) | (CO_df['hour'] <= 15)]

# Shift dates for hours >=21 to next day to group correctly
filtered['group_date'] = (filtered['data'] + pd.to_timedelta((filtered['hour'] >= 21).astype(int), unit='D')).dt.date

# Compute daily mean grouped by the adjusted date
daily_CO = filtered.groupby(['group_date', 'idsensore']).agg({'valore': 'mean'}).reset_index()

# Rename 'group_date' to 'date' if needed
daily_CO.rename(columns={'group_date': 'date'}, inplace=True)

daily_CO

/tmp/ipykernel_74878/349499255.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['group_date'] = (filtered['data'] + pd.to_timedelta((filtered['hour'] >= 21).astype(int), unit='D')).dt.date


,date,idsensore,valore
0,2019-01-01,5812,0.618750
1,2019-01-01,5814,1.243750
2,2019-01-01,5827,1.400000
3,2019-01-01,5834,1.812500
4,2019-01-01,5835,0.825000
...,...,...,...
10608,2025-01-01,5812,1.100000
10609,2025-01-01,5814,1.433333
10610,2025-01-01,5827,2.833333
10611,2025-01-01,5834,1.066667


In [ ]:
daily_CO.to_csv('/path/to/arpa_mean_21_15.csv')